In [1]:
import pandas as pd
import numpy as np 
import sys
import re
sys.path.insert(0, "modules")

import numpy as np
import pandas as pd

import plotly.express as px

import matplotlib.pyplot as plt   

import plotly.graph_objs as go
from plotly.subplots import make_subplots


In [2]:
merged_data=pd.read_csv('data/merged_data.csv')

merged_data.head()

,ISIN,Name,Mkt Cap,Industry,BI350,BI493,EG056,EG057,ES018,ES019,...,SR06420,SR06421,SR06422,SR06423,SR08418,SR08419,SR08420,SR08421,SR08422,SR08423
0,CNE100004ZD8,ACM RESEARCH S-A,"4,885,443,926.00",Semiconductors,0.0,No data,25.644,2.219,No data,No data,...,7.086886,7.086886,7.086886,7.086886,7.334174,7.334174,7.334174,7.334174,7.334174,7.334174
1,US00108J1097,ACM RESEARCH-A,"1,474,910,952.00",Semiconductors,0.0,No data,120.409,47.224,No data,No data,...,2.720348,2.720348,2.720348,2.720348,8.193265,8.193265,8.193265,8.193265,8.193265,8.193265
2,US0079031078,ADV MICRO DEVICE,"241,546,435,400.00",Semiconductors,0.0,No data,385.733,37.01,No data,No data,...,7.079517,7.079517,7.079517,7.079517,8.231827,8.231827,8.231827,8.231827,8.231827,8.231827
3,TW0003438008,ADVANCED ANALOG,"98,788,154.60",Semiconductors,0.0,No data,33.397,7.594,No data,No data,...,7.377765,7.377765,7.377765,7.377765,7.552586,7.552586,7.552586,7.552586,7.552586,7.552586
4,CNE100003MM9,ADVANCED MICRO-A,"11,573,338,720.00",Semiconductors,0.0,100,64.797,4.375,No data,0.023,...,7.086886,7.086886,7.086886,7.086886,6.040477,6.040477,6.040477,6.040477,6.040477,6.040477


In [3]:
df_materiality=pd.read_csv('data/materiality.csv')


In [4]:
df_materiality = df_materiality.fillna('0')
df_materiality = df_materiality.drop('Field Name', axis=1)


In [5]:
for column in df_materiality.columns:
    try:
        df_materiality[column] = pd.to_numeric(df_materiality[column], errors='raise')
    except ValueError:
        pass 

df_materiality.head()

,Type,Issue,Score Field ID,Exploration & Production,Integrated Oils,Midstream,Refining & Marketing,Drilling & Drilling Support,Oilfield Services & Equipment,"Internet Media, Platforms and Services",...,Wind Energy Equipment,Renewable Energy Project Developers,"Technology Hardware, Equipment and Components",EMS/ODM,Semiconductors,Software and Tech Services,Integrated Electric Utilities and Power Generation,Electric Transmission & Dist,Gas Utilities,Water Utilities
0,Environmental,Air Quality,SR208,Y,Y,Y,Y,Y,Y,0,...,0,0,0,0,0.0,0,Y,0,0,0
1,Environmental,Climate Exposure,SR235,Y,Y,0,0,0,0,0,...,0,0,0,0,0.0,0,Y,Y,0,Y
2,Environmental,Ecological Impact,SR231,Y,Y,Y,Y,Y,Y,0,...,Y,Y,0,0,0.0,0,Y,0,Y,Y
3,Environmental,Energy Management,SR205,0,Y,0,0,Y,Y,Y,...,0,0,0,0,0.3,Y,0,0,0,Y
4,Environmental,Environmental Supply Chain Management,SR249,0,0,0,0,0,0,0,...,Y,0,0,0,0.0,0,0,0,0,0


In [6]:
categories = df_materiality['Issue'].tolist()

# Crear un nuevo DataFrame vacío para almacenar los resultados
results = pd.DataFrame(columns=['Industry', 'type', 'ISIN'] + categories)

# Iterar a través de merged_data
for index, row in merged_data.iterrows():
    industry = row['Industry']
    isin = row['ISIN']
    
    # Crear filas base para los resultados
    result_row_env = [industry, 'Environmental', isin] + [0]*len(categories)
    result_row_soc = [industry, 'Social', isin] + [0]*len(categories)
    result_row_gov = [industry, 'Governance', isin] + [0]*len(categories)

    for index2, row2 in df_materiality.iterrows():
        score_id = row2['Score Field ID']
        category_type = row2['Type']
        issue = row2['Issue']
        
        if score_id in row.index:
            try:
                value = float(row[score_id])
            except ValueError:
                value = 0
            
            # Asignar valores a las categorías correspondientes
            if category_type == 'Environmental':
                result_row_env[3 + categories.index(issue)] = value
            elif category_type == 'Social':
                result_row_soc[3 + categories.index(issue)] = value
            elif category_type == 'Governance':
                result_row_gov[3 + categories.index(issue)] = value

    # Convertir listas a Series para agregar al DataFrame de resultados
    result_row_env = pd.Series(result_row_env, index=['Industry', 'type', 'ISIN'] + categories)
    result_row_soc = pd.Series(result_row_soc, index=['Industry', 'type', 'ISIN'] + categories)
    result_row_gov = pd.Series(result_row_gov, index=['Industry', 'type', 'ISIN'] + categories)

    # Agregar las filas al DataFrame de resultados usando concat
    results = pd.concat([results, result_row_env.to_frame().T, result_row_soc.to_frame().T, result_row_gov.to_frame().T], ignore_index=True)


results

,Industry,type,ISIN,Air Quality,Climate Exposure,Ecological Impact,Energy Management,Environmental Supply Chain Management,GHG Emissions Management,Sustainable Finance,...,Labor & Employment Practices,Marketing & Labeling,Occupational Health & Safety Management,Operational Risk Management,Product Quality Management,Social Supply Chain Management,Board Composition,Executive Compensation,Shareholder Rights,Audit
0,Semiconductors,Environmental,CNE100004ZD8,0.0,0.0,0.0,4.816013,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,Semiconductors,Social,CNE100004ZD8,0,0,0,0,0,0,0,...,0.0,0.0,3.0,0.0,0.0,0.0,0,0,0,0
2,Semiconductors,Governance,CNE100004ZD8,0,0,0,0,0,0,0,...,0,0,0,0,0,0,4.781484,1.60677,7.086886,7.334174
3,Semiconductors,Environmental,US00108J1097,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,Semiconductors,Social,US00108J1097,0,0,0,0,0,0,0,...,0.0,0.0,3.0,0.0,0.0,0.0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
988,Semiconductors,Social,US29355A1079,0,0,0,0,0,0,0,...,0.0,0.0,6.123097,0.0,0.0,0.0,0,0,0,0
989,Semiconductors,Governance,US29355A1079,0,0,0,0,0,0,0,...,0,0,0,0,0,0,6.368159,7.543227,2.754029,9.261718
990,Semiconductors,Environmental,US3364331070,0.0,0.0,0.0,5.155654,0.0,8.811409,0.0,...,0,0,0,0,0,0,0,0,0,0
991,Semiconductors,Social,US3364331070,0,0,0,0,0,0,0,...,0.0,0.0,5.460928,0.0,0.0,0.0,0,0,0,0


In [7]:
for index, row in results.iterrows():
    industry = row['Industry']
    
    # Verificar si industry no es NaN
    if pd.notna(industry):
        # Acceder al multiplicador solo si industry es válido
        if industry in df_materiality.columns:
            multiplier = df_materiality[industry].values
            
            # Identificar las columnas a multiplicar (ajustar los nombres de columna según la estructura real de tu DataFrame)
            columns_to_multiply = results.columns[3:]  # Ajustar para seleccionar las columnas que deseas multiplicar
            
            # Realizar la multiplicación para las columnas seleccionadas
            results.loc[index, columns_to_multiply] *= multiplier
        else:
            print(f"Industry '{industry}' no está en df_materiality.")
    else:
        print("Warning: 'Industry' value is NaN for row index", index)
results
        

,Industry,type,ISIN,Air Quality,Climate Exposure,Ecological Impact,Energy Management,Environmental Supply Chain Management,GHG Emissions Management,Sustainable Finance,...,Labor & Employment Practices,Marketing & Labeling,Occupational Health & Safety Management,Operational Risk Management,Product Quality Management,Social Supply Chain Management,Board Composition,Executive Compensation,Shareholder Rights,Audit
0,Semiconductors,Environmental,CNE100004ZD8,0.0,0.0,0.0,1.444804,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Semiconductors,Social,CNE100004ZD8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.08,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Semiconductors,Governance,CNE100004ZD8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.721334,0.385625,1.700853,1.173468
3,Semiconductors,Environmental,US00108J1097,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Semiconductors,Social,US00108J1097,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.08,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
988,Semiconductors,Social,US29355A1079,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.204315,0.0,0.0,0.0,0.0,0.0,0.0,0.0
989,Semiconductors,Governance,US29355A1079,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.292537,1.810374,0.660967,1.481875
990,Semiconductors,Environmental,US3364331070,0.0,0.0,0.0,1.546696,0.0,1.145483,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
991,Semiconductors,Social,US3364331070,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.965934,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:

results['ENVIRONMENTAL SCORE'] = results[results['type'] == 'Environmental'].iloc[:, 3:].sum(axis=1) * 0.4

results['SOCIAL SCORE'] = results[results['type'] == 'Social'].iloc[:, 3:].sum(axis=1) * 0.3


results['GOVERNANCE SCORE'] = results[results['type'] == 'Governance'].iloc[:, 3:].sum(axis=1) * 0.3

results['GLOBAL SCORE'] = results.groupby('ISIN')[['ENVIRONMENTAL SCORE', 'SOCIAL SCORE', 'GOVERNANCE SCORE']].transform('sum').sum(axis=1)

In [9]:
results

,Industry,type,ISIN,Air Quality,Climate Exposure,Ecological Impact,Energy Management,Environmental Supply Chain Management,GHG Emissions Management,Sustainable Finance,...,Product Quality Management,Social Supply Chain Management,Board Composition,Executive Compensation,Shareholder Rights,Audit,ENVIRONMENTAL SCORE,SOCIAL SCORE,GOVERNANCE SCORE,GLOBAL SCORE
0,Semiconductors,Environmental,CNE100004ZD8,0.0,0.0,0.0,1.444804,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.227204,NaN,NaN,3.045588
1,Semiconductors,Social,CNE100004ZD8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.324,NaN,3.045588
2,Semiconductors,Governance,CNE100004ZD8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.721334,0.385625,1.700853,1.173468,NaN,NaN,1.494384,3.045588
3,Semiconductors,Environmental,US00108J1097,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,2.030903
4,Semiconductors,Social,US00108J1097,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.324,NaN,2.030903
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
988,Semiconductors,Social,US29355A1079,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.751295,NaN,4.590477
989,Semiconductors,Governance,US29355A1079,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.292537,1.810374,0.660967,1.481875,NaN,NaN,1.873726,4.590477
990,Semiconductors,Environmental,US3364331070,0.0,0.0,0.0,1.546696,0.0,1.145483,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.832406,NaN,NaN,4.511299
991,Semiconductors,Social,US3364331070,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.67978,NaN,4.511299


In [10]:
results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 993 entries, 0 to 992
Data columns (total 32 columns):
 #   Column                                   Non-Null Count  Dtype 
---  ------                                   --------------  ----- 
 0   Industry                                 993 non-null    object
 1   type                                     993 non-null    object
 2   ISIN                                     993 non-null    object
 3   Air Quality                              993 non-null    object
 4   Climate Exposure                         993 non-null    object
 5   Ecological Impact                        993 non-null    object
 6   Energy Management                        993 non-null    object
 7   Environmental Supply Chain Management    993 non-null    object
 8   GHG Emissions Management                 993 non-null    object
 9   Sustainable Finance                      993 non-null    object
 10  Sustainable Product                      993 non-null    objec

In [11]:
results['ENVIRONMENTAL SCORE'] = results['ENVIRONMENTAL SCORE'].fillna(0)
results['SOCIAL SCORE'] = results['SOCIAL SCORE'].fillna(0)
results['GOVERNANCE SCORE'] = results['GOVERNANCE SCORE'].fillna(0)
results['GLOBAL SCORE'] = results['GLOBAL SCORE'].fillna(0)
results

C:\Users\Veronica\AppData\Local\Temp\ipykernel_28348\3064771603.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  results['ENVIRONMENTAL SCORE'] = results['ENVIRONMENTAL SCORE'].fillna(0)
C:\Users\Veronica\AppData\Local\Temp\ipykernel_28348\3064771603.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  results['SOCIAL SCORE'] = results['SOCIAL SCORE'].fillna(0)
C:\Users\Veronica\AppData\Local\Temp\ipykernel_28348\3064771603.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version.

,Industry,type,ISIN,Air Quality,Climate Exposure,Ecological Impact,Energy Management,Environmental Supply Chain Management,GHG Emissions Management,Sustainable Finance,...,Product Quality Management,Social Supply Chain Management,Board Composition,Executive Compensation,Shareholder Rights,Audit,ENVIRONMENTAL SCORE,SOCIAL SCORE,GOVERNANCE SCORE,GLOBAL SCORE
0,Semiconductors,Environmental,CNE100004ZD8,0.0,0.0,0.0,1.444804,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.227204,0.000000,0.000000,3.045588
1,Semiconductors,Social,CNE100004ZD8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.324000,0.000000,3.045588
2,Semiconductors,Governance,CNE100004ZD8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.721334,0.385625,1.700853,1.173468,0.000000,0.000000,1.494384,3.045588
3,Semiconductors,Environmental,US00108J1097,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,2.030903
4,Semiconductors,Social,US00108J1097,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.324000,0.000000,2.030903
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
988,Semiconductors,Social,US29355A1079,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.751295,0.000000,4.590477
989,Semiconductors,Governance,US29355A1079,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.292537,1.810374,0.660967,1.481875,0.000000,0.000000,1.873726,4.590477
990,Semiconductors,Environmental,US3364331070,0.0,0.0,0.0,1.546696,0.0,1.145483,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.832406,0.000000,0.000000,4.511299
991,Semiconductors,Social,US3364331070,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.679780,0.000000,4.511299


In [14]:
for column in results.columns:
    try:
        results[column] = pd.to_numeric(results[column], errors='raise')
    except ValueError:
        pass 

results.head()

,Industry,type,ISIN,Air Quality,Climate Exposure,Ecological Impact,Energy Management,Environmental Supply Chain Management,GHG Emissions Management,Sustainable Finance,...,Product Quality Management,Social Supply Chain Management,Board Composition,Executive Compensation,Shareholder Rights,Audit,ENVIRONMENTAL SCORE,SOCIAL SCORE,GOVERNANCE SCORE,GLOBAL SCORE
0,Semiconductors,Environmental,CNE100004ZD8,0.0,0.0,0.0,1.444804,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,1.227204,0.000,0.000000,3.045588
1,Semiconductors,Social,CNE100004ZD8,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.324,0.000000,3.045588
2,Semiconductors,Governance,CNE100004ZD8,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,1.721334,0.385625,1.700853,1.173468,0.000000,0.000,1.494384,3.045588
3,Semiconductors,Environmental,US00108J1097,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,2.030903
4,Semiconductors,Social,US00108J1097,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.324,0.000000,2.030903


In [ ]:
results.to_csv('data/results.csv', index=False)

In [26]:
sorted(results['GLOBAL SCORE'].unique())

[0.0,
 0.6697712825759999,
 1.0237940311679998,
 1.1119198236119998,
 1.1985766124759998,
 1.3060659742679999,
 1.3643999576999999,
 1.388522417988,
 1.453707584328,
 1.4916449944199999,
 1.5051490202399997,
 1.505703624204,
 1.5127821664799999,
 1.538070401592,
 1.60271897508,
 1.6154462900039999,
 1.636391080848,
 1.6368696670560001,
 1.74861304632,
 1.7499389218919998,
 1.7825630608199998,
 1.7944366454879996,
 1.81770709926,
 1.8557714081039998,
 1.8594723987839994,
 1.8963868643159998,
 1.908408344592,
 1.9086193313880002,
 1.931112086936,
 1.94168769234,
 2.0016609440879995,
 2.007445583868,
 2.009061492288,
 2.0223786382439997,
 2.024230427424,
 2.024587014112,
 2.030903148648,
 2.056396848084,
 2.0625147195439997,
 2.066726946504,
 2.0757334722599996,
 2.080769473992,
 2.0840441971440002,
 2.087724503472,
 2.0925680656799996,
 2.11014737568,
 2.11067176725,
 2.126039062548,
 2.1305217552,
 2.158177321776,
 2.162972685192,
 2.2068960424800004,
 2.2491662087399997,
 2.25245342439

In [22]:
cols_to_average = results.columns[3:]
mean_results = results.groupby('Industry')[cols_to_average].mean()
mean_results

,Air Quality,Climate Exposure,Ecological Impact,Energy Management,Environmental Supply Chain Management,GHG Emissions Management,Sustainable Finance,Sustainable Product,Waste Management,Water Management,...,Product Quality Management,Social Supply Chain Management,Board Composition,Executive Compensation,Shareholder Rights,Audit,ENVIRONMENTAL SCORE,SOCIAL SCORE,GOVERNANCE SCORE,GLOBAL SCORE
Industry,,,,,,,,,,,,,,,,,,,,,
Semiconductors,0.0,0.0,0.0,0.322445,0.0,0.122011,0.0,0.041088,0.115762,0.380495,...,0.120133,0.0,0.585272,0.355444,0.528436,0.371093,0.39272,0.310146,0.552073,3.764817


In [27]:
cols_to_sum = results.columns[3:-1]  # Excluye las primeras 3 columnas y 'GLOBAL SCORE'

sum_results = results.groupby('Industry')[cols_to_sum].sum()

unique_isins_per_industry = results.groupby('Industry')['ISIN'].nunique()

mean_results = sum_results.div(unique_isins_per_industry, axis=0)

mean_results['GLOBAL SCORE'] = results.groupby('Industry')['GLOBAL SCORE'].mean()

In [28]:
mean_results

,Air Quality,Climate Exposure,Ecological Impact,Energy Management,Environmental Supply Chain Management,GHG Emissions Management,Sustainable Finance,Sustainable Product,Waste Management,Water Management,...,Product Quality Management,Social Supply Chain Management,Board Composition,Executive Compensation,Shareholder Rights,Audit,ENVIRONMENTAL SCORE,SOCIAL SCORE,GOVERNANCE SCORE,GLOBAL SCORE
Industry,,,,,,,,,,,,,,,,,,,,,
Semiconductors,0.0,0.0,0.0,0.967335,0.0,0.366032,0.0,0.123263,0.347286,1.141484,...,0.360398,0.0,1.755816,1.066333,1.585308,1.113278,1.17816,0.930437,1.65622,3.764817


In [30]:
mean_results.to_csv('data/mean_results.csv', index=False)